In [1]:
import pandas as pd
import numpy as np

In [2]:
temp_df_pt1 = pd.read_csv('../actual_temperature_data/gridded_averages_1880-2021/NOAAGlobalTemp_v5a_1.csv')
temp_df_pt2 = pd.read_csv('../actual_temperature_data/gridded_averages_1880-2021/NOAAGlobalTemp_v5a_2.csv')

# concatenate dataframes
# gonna turn all the wrong years into -1
frames = [temp_df_pt1, temp_df_pt2]
temp_df = pd.concat(frames, sort=False)

tree_df = pd.read_csv('../tree_data.csv')
tree_df['year'] = tree_df['year'].apply(lambda x: float(x) if str(x).isdigit() else -1)
tree_df.head()

C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,year,avg_tree_ring_width,lat,long,elevation,tree_species,organism_group
0,969.0,164.5,51.75041,-0.34223,NaN,Quercus spp. L.,0
1,970.0,188.5,51.75041,-0.34223,NaN,Quercus spp. L.,0
2,971.0,139.0,51.75041,-0.34223,NaN,Quercus spp. L.,0
3,972.0,173.0,51.75041,-0.34223,NaN,Quercus spp. L.,0
4,973.0,221.0,51.75041,-0.34223,NaN,Quercus spp. L.,0


In [13]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3103094 entries, 340 to 1307346
Data columns (total 6 columns):
time    object
lat     float64
lon     float64
z       float64
anom    float64
year    object
dtypes: float64(4), object(2)
memory usage: 245.7+ MB


In [3]:
# How many have bad years
print(tree_df.shape)
tree_df[tree_df['year'] == -1].shape

(2062155, 7)


(120637, 7)

In [4]:
def get_5x5_grid(og_lat, og_lon, base=2.5):
    lat1 = base * round((og_lat-2.5)/base)
    lat2 = base * round((og_lat+2.5)/base)

    lon1 = base * round((og_lon-2.5)/base)
    lon2 = base * round((og_lon+2.5)/base)

    if lat2-2.5 < og_lat:
        lat2 += 2.5
        lat1 += 2.5
    else:
        lat2 -= 2.5
        lat1 -= 2.5

    if lon2-2.5 < og_lon:
        lon2 += 2.5
        lon1 += 2.5
    else:
        lon2 -= 2.5
        lon1 -= 2.5

    if (lat1 % 5 )== 0 or (lat2 % 5 )== 0:
        if lat1 < 0:
            lat1 += 2.5
            lat2 += 2.5
        else: 
            lat1 -= 2.5
            lat2 -= 2.5

    if (lon1 % 5 )== 0 or (lon2 % 5 )== 0:
        lon1 += 2.5
        lon2 += 2.5

    if (lat1 == -92.5 and lat2 == -87.5):
        lat1 = -87.5
        lat2 = -82.5
    elif (lat1 == 87.5 and lat2 == 92.5):
        lat1 = 82.5
        lat2 = 87.5
    
    if (lon1 == 0.0 and lon2 == 5.0):
        lon1 = 2.5
        lon2 = 7.5
    elif (lon1 == 355.0 and lon2 == 360.0):
        lon1 = 352.5
        lon2 = 357.5

    return lat1, lat2, lon1, lon2

In [5]:
temp_df['year'] = temp_df['time'].apply(lambda x: str(x)[0:4])
temp_df = temp_df.dropna(subset=['anom'])
temp_annual_df =  temp_df.groupby(['year', 'lat', 'lon']).agg({'anom': ['mean']}).reset_index()

In [14]:
# given an observation row, return the row with the correct 5x5 temperature anomaly

def get_gridded_temp_anom(lat, lon, year):
    lat1, lat2, lon1, lon2 = get_5x5_grid(lat, lon)
    print('a')
    temp_df_for_lat_lon = temp_df[((temp_df['lat'] == lat1) | (temp_df['lat'] == lat2)) &
                                  ((temp_df['lon'] == lon1) | (temp_df['lon'] == lon2)) &
                                  (temp_df['year'] == str(year))]
    print('b')
    annual_temp_df_for_lat_lon = temp_df_for_lat_lon.groupby(['year']).agg({'anom': ['mean']}).reset_index()
    print('c')
    if len(annual_temp_df_for_lat_lon[annual_temp_df_for_lat_lon['year']==str(year)]['anom']['mean'].tolist()) > 0:
        temp_anom = annual_temp_df_for_lat_lon[annual_temp_df_for_lat_lon['year']==str(year)]['anom']['mean'].tolist()[0]
        return temp_anom

In [18]:
# run it on the data

tree_df_temp = tree_df[tree_df['year'] > 1879]
tree_df_temp = tree_df_temp[tree_df_temp['year'] < 2021]
tree_df_temp = tree_df_temp.iloc[:10,:]
tree_df_temp['temp_anom'] = tree_df.iloc[:10,:].apply(lambda x: get_gridded_temp_anom(x['lat'], x['long'], x['year']), axis=1)
tree_df_temp

a
b
c
a
b
c
a
b
c
a
b
c
a
b
c
a
b
c
a
b
c
a
b
c
a
b
c
a
b
c


,year,avg_tree_ring_width,lat,long,elevation,tree_species,organism_group,temp_anom
454,1880.0,122.939394,37.42,30.28,1601.0,Pinus nigra J.F. Arnold,0,NaN
455,1881.0,140.909091,37.42,30.28,1601.0,Pinus nigra J.F. Arnold,0,NaN
456,1882.0,129.121212,37.42,30.28,1601.0,Pinus nigra J.F. Arnold,0,NaN
457,1883.0,127.666667,37.42,30.28,1601.0,Pinus nigra J.F. Arnold,0,NaN
458,1884.0,132.393939,37.42,30.28,1601.0,Pinus nigra J.F. Arnold,0,NaN
459,1885.0,160.000000,37.42,30.28,1601.0,Pinus nigra J.F. Arnold,0,NaN
460,1886.0,142.969697,37.42,30.28,1601.0,Pinus nigra J.F. Arnold,0,NaN
461,1887.0,98.090909,37.42,30.28,1601.0,Pinus nigra J.F. Arnold,0,NaN
462,1888.0,115.818182,37.42,30.28,1601.0,Pinus nigra J.F. Arnold,0,NaN
463,1889.0,159.424242,37.42,30.28,1601.0,Pinus nigra J.F. Arnold,0,NaN
